In [0]:
'''
lms 강의 자료 다운로드 프로그램
(lms.koipa.or.kr의 강의 자료를 저장하기 위한 프로그램입니다.)

Copyright (c) 2019, PARKHOJEONG 
All rights reserved. 

-------------------------------------< 설명서 >-------------------------------
[주의 사항]
다운로드 이후 다운로드 받은 파일이나 폴더를 사용하고 있다면 재다운로드시 작동이 되지 않습니다.
기본 사용 방법을 읽고 정확하게 따라해 주세요.

[기본 사용 방법]
- 다운로드한 파일을 원하는 폴더 내부에 위치시킨다.(예를 들면 "ai_기본과정" 폴더를 만든 후 이 폴더에 파일을 위치시킨다.)

- 다운로드한 파일을 아나콘다의 주피터 노트북으로 open 한다.

- 로그인을 위한 아이디와 비밀번호를 입력한다.(본인의 로그인 정보를 LOGIN_EMAIL과 LOGIN_PASSWORD에 대입하세요.)

'''

LOGIN_EMAIL      = "@gmail.com" # str 타입으로 입력하세요. 로그인 아이디(이메일) 
LOGIN_PASSWORD   = ""        # str 타입으로 입력하세요. 비밀번호

'''
- 실행한다.

- 추가 조작을 원하시면 아래의 <상세 설명서> 부분을 참조하세요.

- 실행이 안된다면 다음 문제를 해결해주세요.

문제 원인 1)

로그인 아이디와 비밀번호를 확인해주세요.


문제 원인 2)
다음 모듈 임포트 구분을 수행할 수 있도록 아나콘다 prompt에서 pip를 이용해 설치해주세요. 

import requests
from bs4 import BeautifulSoup

위 문제점을 해결하고도 작동이 안 된다면 parkhj062@gmail.com 로 문의해주세요.

----------------------------------------------------------------------------



--------------------------------------<상세 설명서>---------------------------------
[사용 방법]
1. 로컬 폴더에 파일을 다운로드한다. 혹은 코드를 복사한다. (파일명은 달라도 됨.)
2. 파일을 아나콘다 주피터 노트북으로 연다.
3. 사용자 입력 부분을 입력한다. (다운로드 처음 수업 회차, 다운로드 마지막 수업 회차를 입력한다.)
4. 이 파일을 주피터로 실행한다. (shift+ enter)


사용 예시는 다음과 같습니다.

예시 1)
2019-07-01 ~ 2017-07-05 에 대한 강의자료 다운로드하는 경우
수업 1번째 날부터 5번째 수업까지 다운로드 받고 싶다면 

start_date_order = 1
total_date_order = 5 으로 변경하세요.


예시 2)
2019-07-26 ~ 2019-07-26 에 대한 강의자료 다운로드하는 경우
현재가 20번째 수업이고 수업 도중 한 번씩 다운로드 하고 싶은 경우.

start_date_order = 20
total_date_order = 20 


예시 3)
2019-07-01 ~ 2019-08-31 에 대한 강의자료 다운로드하는 경우
모든 수업이 끝난 후 다운로드를 받고 싶다면 1, 40 으로 써주시면 됩니다.

start_date_order = 1
total_date_order = 40 

'''

# 사용자 입력 부분

start_date_order = 1          # int 타입으로 입력하세요. 다운로드 처음 수업 회차
total_date_order = 40         # int 타입으로 입력하세요. 다운로드 마지막 수업 회차

#------------------------------------------------------------------------------------

# 실행 코드

print("로그인 정보 => email: {} , password: {} ".format(LOGIN_EMAIL, LOGIN_PASSWORD))
print("{}번째 수업부터 {}번째 수업까지 다운로드 됩니다. ".format(start_date_order, total_date_order))
from pprint import pprint 
import os
import shutil
import platform

try:
    import requests
    from bs4 import BeautifulSoup

    print('\nall Module was installed')
except ImportError as err:
    print('\n모듈 설치가 필요합니다. =>' )
    raise Exception(err)

 
OS_NAME = platform.system()

# session created 
# because site does not return cookie after login process
s = requests.Session()

# login request url, login data 
BASE_URL = "https://lms.koipa.or.kr"
LOGIN_URL = BASE_URL+"/auth/login"
data = {'email': LOGIN_EMAIL, 'password': LOGIN_PASSWORD, 'next': ''}

# request return value
res = s.post(LOGIN_URL, data, verify=False)

# request check
print(res.url)

# html to parse 
class_html = s.get(BASE_URL+"/course/17")


if OS_NAME == 'Windows': # WINDOWS
  soup_course = BeautifulSoup(class_html.text, 'html5lib')
elif OS_NAME == 'Darwin': # MAC
  soup_course = BeautifulSoup(class_html.text, 'html5lib')
else:
  raise Exception("OS["+OS_NAME+"] is not supported ")

soup_course.prettify()

# login pass check
try :
    soup_course.find_all('tbody')[1] 
except Exception as err:
    print(err)
    print("로그인 정보를 확인하세요")
    
    
# start index 0 in python
start_date_order -= 1

# download files
for date_order in range(start_date_order, total_date_order):
    
    # parse tbody
    tbody_tag_obj = soup_course.find_all('tbody')[date_order]
    
    # parse date for folder name
    date = tbody_tag_obj.find(class_='border_bottom border_top')
    folder_name = date.attrs['id']
    folder_content = tbody_tag_obj.find_all('td')[2].find('span').text
    
    print(folder_name, folder_content)
    
    # eliminate escape character 
    folder_content = "".join([x for x in folder_content if x.isalpha()])
    
    # folder name
    folder_name = folder_name + " " + folder_content 
    print(folder_name)
    
    # parsing for link and file name

    link_list = []
    title_list = []
    
    a_tag_obj_list = tbody_tag_obj.find_all('a')
    
    for i,u in enumerate(a_tag_obj_list):
        
      if i >=1 :
        link_list.append(BASE_URL+u.attrs['href'])
        title_list.append(u.text[1:])

    # eliminate existed directory
    try:
        shutil.rmtree(folder_name)
#         print("폴더 "+folder_name+"이 삭제되었습니다.")
    except Exception as err:
        pass
    
    # make new directory
    os.mkdir(folder_name)
#     print("폴더" +folder_name+"이 생성되었습니다.")
    
    # download in directory
    for index in range(len(link_list)):
        cont = s.get(link_list[index])
        title_name = title_list[index]
        
        if OS_NAME == "Windows" :
            dir_name = folder_name +'\\' + title_name
        elif OS_NAME == "Darwin" :
            dir_name = folder_name +'/' + title_name
        else:
            raise Exception(OS_NAME +" is not supported ")
        
        with open(dir_name, "wb") as f:
            f.write(cont.content)
              
        
                
    

print("실행 완료")

로그인 정보 => email: parkhj062@gmail.com , password: 12345678 
1번째 수업부터 32번째 수업까지 다운로드 됩니다. 

all Module was installed


/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


https://lms.koipa.or.kr/dashboard/


/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2019-07-01 과정 소개 및 파이썬 DataType
2019-07-01 과정소개및파이썬DataType


...

2019-08-19 텐서플로우 (IV)
2019-08-19 텐서플로우IV


/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is be

2019-08-20 텐서플로우 (V)
2019-08-20 텐서플로우V


/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


실행 완료
